# 🔧 Generación Automática de Código ETL con LLMs

Objetivo: automatizar la creación de pipelines ETL, transformaciones y scripts de datos usando IA generativa, con validación y best practices.

- Duración: 90-120 min
- Dificultad: Media/Alta
- Prerrequisitos: GenAI 01-02, experiencia con ETL

## 1. Generación de función ETL básica

In [ ]:
import os
from openai import OpenAI
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

def generate_etl_code(description: str) -> str:
    prompt = f'''
Eres un experto en Python y pipelines ETL. Genera código Python completo y ejecutable.

Requerimientos:
{description}

Incluye:
- Imports necesarios
- Manejo de errores
- Logging básico
- Docstrings
- Type hints

Código:
'''
    resp = client.chat.completions.create(
        model='gpt-4',
        messages=[{'role':'user','content':prompt}],
        temperature=0.2
    )
    return resp.choices[0].message.content.strip().replace('```python','').replace('```','')

desc = '''
Función que:
1. Lee CSV de ventas desde S3 (boto3)
2. Filtra filas con total > 0
3. Agrega columna "mes" (YYYY-MM) desde "fecha"
4. Escribe a Parquet particionado por mes
'''

code = generate_etl_code(desc)
print(code)

## 2. Generación de transformación Pandas

In [ ]:
transformation_spec = '''
Dataset: transacciones bancarias (CSV)
Columnas: trans_id, fecha, monto, tipo (debito/credito), cuenta_id

Transformaciones:
1. Convertir fecha a datetime
2. Crear columna "anio_mes" (formato YYYY-MM)
3. Calcular saldo acumulado por cuenta_id ordenado por fecha
4. Agregar flag is_anomaly si monto > 3 desviaciones estándar de la media de esa cuenta
5. Exportar a CSV con encoding UTF-8
'''

transform_code = generate_etl_code(transformation_spec)
print(transform_code[:500] + '...')  # Preview

## 3. Validación de código generado

In [ ]:
import ast
import subprocess

def validate_python_syntax(code: str) -> tuple[bool, str]:
    """Valida sintaxis Python sin ejecutar."""
    try:
        ast.parse(code)
        return True, 'Sintaxis válida'
    except SyntaxError as e:
        return False, f'Error de sintaxis: {e}'

def lint_code(code: str, tool='flake8') -> str:
    """Ejecuta linter (requiere instalado)."""
    try:
        with open('/tmp/temp_code.py', 'w') as f:
            f.write(code)
        result = subprocess.run([tool, '/tmp/temp_code.py'], capture_output=True, text=True)
        return result.stdout if result.returncode == 0 else result.stderr
    except Exception as e:
        return f'Error linting: {e}'

valid, msg = validate_python_syntax(code)
print(f'{"✅" if valid else "❌"} {msg}')

## 4. Generación de DAG de Airflow

In [ ]:
dag_spec = '''
DAG de Airflow para:
- Nombre: ventas_daily_etl
- Schedule: diario a las 2 AM
- Tareas:
  1. extract_s3: descarga ventas.csv de S3
  2. validate: Great Expectations (columnas no nulas, total > 0)
  3. transform: agrega mes, calcula métricas
  4. load_db: inserta en PostgreSQL (tabla ventas_daily)
  5. send_alert: email si falla cualquier paso
- Retries: 2 con delay de 5 min
'''

dag_code = generate_etl_code(dag_spec)
print(dag_code[:600] + '...')

## 5. Iteración y mejora con feedback

In [ ]:
def refine_code(original_code: str, feedback: str) -> str:
    prompt = f'''
Mejora este código Python según el feedback:

Código original:
{original_code}

Feedback:
{feedback}

Código mejorado:
'''
    resp = client.chat.completions.create(
        model='gpt-4',
        messages=[{'role':'user','content':prompt}],
        temperature=0.1
    )
    return resp.choices[0].message.content.strip().replace('```python','').replace('```','')

feedback_example = '''
- Añadir retry con exponential backoff en la descarga de S3
- Usar context manager para conexión a DB
- Loggear número de filas procesadas
'''

improved = refine_code(code, feedback_example)
print(improved[:400] + '...')

## 6. Generación de tests unitarios

In [ ]:
def generate_tests(code: str) -> str:
    prompt = f'''
Genera tests unitarios con pytest para este código:

{code}

Incluye:
- Test de caso normal (happy path)
- Test con datos vacíos
- Test con errores (valores nulos, tipos incorrectos)
- Mocks para I/O externo (S3, DB)

Tests:
'''
    resp = client.chat.completions.create(
        model='gpt-4',
        messages=[{'role':'user','content':prompt}],
        temperature=0.1
    )
    return resp.choices[0].message.content.strip().replace('```python','').replace('```','')

test_code = generate_tests(code)
print(test_code[:500] + '...')

## 7. Buenas prácticas

- **Revisar siempre**: nunca ejecutes código generado sin inspección humana.
- **Validación automática**: sintaxis, linting, tests.
- **Versionado**: guarda código generado en Git con mensaje descriptivo.
- **Plantillas**: usa templates para estructura consistente.
- **Iteración**: refina con feedback humano y re-generación.
- **Documentación**: genera README y comentarios junto con código.

## 8. Ejercicios

1. Genera un script que migre datos de MongoDB a PostgreSQL.
2. Crea un pipeline Spark (PySpark) que lea Parquet y escriba Delta Lake.
3. Automatiza generación de un data quality report con Great Expectations.
4. Construye un CLI (Click/Typer) generado por LLM para ejecutar ETLs.